In [1]:
import pandas as pd

# Load the CSV file
file_path = "ScanRecords.csv"  # Replace with the actual path to your CSV file
try:
    data = pd.read_csv(file_path)
    print("CSV file loaded successfully!")
    print(data.head())  # Display the first few rows of the DataFrame
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

CSV file loaded successfully!
         Date   Time  Duration PatientType
0  2023-08-01   8.23  0.949176      Type 2
1  2023-08-01   8.49  0.479593      Type 1
2  2023-08-01   9.12  0.496112      Type 2
3  2023-08-01  10.26  0.691947      Type 2
4  2023-08-01  10.64  0.345412      Type 1


In [2]:
# Filter the data for Type 1 patients
type1_data = data[data['PatientType'] == 'Type 1']

# Group by date and count arrivals per day
daily_arrivals = type1_data.groupby('Date').size()
lambda_estimate = daily_arrivals.mean()  # Mean of daily arrivals
print(f"Estimated daily arrival rate (λ): {lambda_estimate:.2f}")

Estimated daily arrival rate (λ): 16.48


In [4]:
# Convert 'Duration' to a numpy array
scan_durations = type1_data['Duration'].values
mu_estimate = scan_durations.mean()  # Mean of scan durations
sigma_estimate = scan_durations.std()  # Standard deviation of scan durations
print(f"Estimated mean (μ): {mu_estimate:.2f}")
print(f"Estimated standard deviation (σ): {sigma_estimate:.2f}")

Estimated mean (μ): 0.43
Estimated standard deviation (σ): 0.10


In [10]:
import numpy as np
from scipy.stats import norm

def bootstrap_parametric(n_bootstraps=1000):
    boot_means = []
    boot_stds = []
    for _ in range(n_bootstraps):
        # Generate bootstrap samples from the fitted normal distribution
        boot_sample = norm.rvs(loc=mu_estimate, scale=sigma_estimate, size=len(scan_durations))
        boot_means.append(boot_sample.mean())
        boot_stds.append(boot_sample.std())
    return np.array(boot_means), np.array(boot_stds)

# Run the bootstrap
n_bootstraps = 1000
boot_means, boot_stds = bootstrap_parametric(n_bootstraps)

# Confidence intervals
mean_ci = (np.percentile(boot_means, 2.5), np.percentile(boot_means, 97.5))
std_ci = (np.percentile(boot_stds, 2.5), np.percentile(boot_stds, 97.5))
print(f"95% CI for mean (μ): {mean_ci}")
print(f"95% CI for standard deviation (σ): {std_ci}")

95% CI for mean (μ): (np.float64(0.4218715866383345), np.float64(0.4423800894831401))
95% CI for standard deviation (σ): (np.float64(0.09033253767634766), np.float64(0.10486509073362885))


In [11]:
# The daily_arrivals mean already serves as the MLE for λ in a Poisson distribution
print(f"MLE for daily arrival rate (λ): {lambda_estimate:.2f}")

MLE for daily arrival rate (λ): 16.48


In [12]:
# Estimate confidence intervals for daily arrival rate
arrival_ci = (np.percentile(daily_arrivals, 2.5), np.percentile(daily_arrivals, 97.5))
print(f"95% CI for daily arrivals: {arrival_ci}")

95% CI for daily arrivals: (np.float64(10.55), np.float64(23.0))


In [13]:
# Print previously calculated CI for scan duration
print(f"95% CI for mean (μ): {mean_ci}")
print(f"95% CI for standard deviation (σ): {std_ci}")

95% CI for mean (μ): (np.float64(0.4218715866383345), np.float64(0.4423800894831401))
95% CI for standard deviation (σ): (np.float64(0.09033253767634766), np.float64(0.10486509073362885))


In [37]:
from scipy.stats import norm
import numpy as np

# Given parameters for Type 1 patients
mean_duration = 0.43  # in hours
std_duration = 0.10   # in hours
confidence_level = 0.99

# Calculate the appropriate timeslot length
z_score = norm.ppf((1 + confidence_level) / 2)  # 99% confidence interval
timeslot_length = mean_duration + z_score * std_duration
print(f"Recommended timeslot length for Type 1 patients (99% CI): {timeslot_length:.2f} hours ({timeslot_length * 60:.2f} minutes)")


Recommended timeslot length for Type 1 patients (99% CI): 0.69 hours (41.25 minutes)


In [38]:
# Simulation parameters
num_days = 30
mean_daily_arrivals = 16.48  # Poisson mean

# Generate simulated data
simulated_data = []
for day in range(num_days):
    # Generate number of arrivals for the day
    daily_arrivals = np.random.poisson(mean_daily_arrivals)
    
    # Generate scan durations for the arrivals
    daily_durations = np.random.normal(loc=mean_duration, scale=std_duration, size=daily_arrivals)
    
    # Store the data
    for scan_duration in daily_durations:
        simulated_data.append({"Day": day + 1, "ScanDuration": scan_duration})

# Convert to a DataFrame for analysis
import pandas as pd

simulated_df = pd.DataFrame(simulated_data)
print(simulated_df.head())  # Preview the simulated data


   Day  ScanDuration
0    1      0.346694
1    1      0.418323
2    1      0.366444
3    1      0.603885
4    1      0.397876


In [42]:
# Evaluate scheduling performance
timeslot_length_minutes = timeslot_length * 60
facility_start_time = 8 * 60  # 8:00 AM in minutes
facility_end_time = 17 * 60  # 5:00 PM in minutes
total_available_time = facility_end_time - facility_start_time

# Calculate daily utilization and overtime
daily_performance = []
for day in simulated_df["Day"].unique():
    day_data = simulated_df[simulated_df["Day"] == day]
    total_scan_time = day_data["ScanDuration"].sum() * 60  # Convert hours to minutes
    
    # Check if overtime occurs
    overtime = max(0, total_scan_time - total_available_time)  # Overtime in minutes
    utilization = total_scan_time / total_available_time  # Utilization as a fraction
    
    daily_performance.append({
        "Day": day,
        "Utilization (fraction)": utilization,  # Unit: Fraction of total available time
        "Overtime (minutes)": overtime  # Unit: Minutes
    })

# Convert to DataFrame for analysis
performance_df = pd.DataFrame(daily_performance)
print(performance_df.describe())  # Summarize the performance metrics



             Day  Utilization (fraction)  Overtime (minutes)
count  30.000000               30.000000           30.000000
mean   15.500000                0.837499           12.491490
std     8.803408                0.192712           29.370365
min     1.000000                0.521637            0.000000
25%     8.250000                0.693493            0.000000
50%    15.500000                0.866001            0.000000
75%    22.750000                0.940615            0.000000
max    30.000000                1.198985          107.452121


In [43]:
performance_df

,Day,Utilization (fraction),Overtime (minutes)
0,1,0.694592,0.000000
1,2,0.993075,0.000000
2,3,0.552139,0.000000
3,4,0.870928,0.000000
4,5,0.928020,0.000000
5,6,0.901034,0.000000
6,7,1.136682,73.808420
7,8,0.901431,0.000000
8,9,0.629642,0.000000
9,10,0.944814,0.000000


In [44]:
import scipy.stats as stats

# Type 2 patient simulation parameters (assuming mean and std from fitted distributions)
mean_arrivals_type2 = 12  # Adjusted mean for Type 2 patient daily arrivals
std_arrivals_type2 = 4  # Adjusted standard deviation for Type 2 patient daily arrivals

# Choose either Gamma or Log-Normal for scan durations based on fit
mean_duration_type2 = 0.55  # Example mean for Type 2 scan duration in hours
std_duration_type2 = 0.12  # Example std for Type 2 scan duration in hours

# Normal distribution for inter-arrival times (example values)
mean_inter_arrival_type2 = 0.15  # Mean inter-arrival time in hours
std_inter_arrival_type2 = 0.05  # Standard deviation for inter-arrival times

# Simulation of Type 2 patient arrivals and scan durations for 30 days
simulated_data_type2 = []
for day in range(num_days):
    daily_arrivals_type2 = np.random.normal(mean_arrivals_type2, std_arrivals_type2)
    daily_arrivals_type2 = max(0, int(np.round(daily_arrivals_type2)))  # Ensure non-negative
    
    # Generate scan durations using Gamma or Log-Normal distribution
    if np.random.choice([True, False]):
        daily_durations_type2 = np.random.gamma(mean_duration_type2, std_duration_type2, daily_arrivals_type2)
    else:
        daily_durations_type2 = np.random.lognormal(np.log(mean_duration_type2), std_duration_type2, daily_arrivals_type2)
    
    # Store the data for Type 2
    for scan_duration in daily_durations_type2:
        simulated_data_type2.append({"Day": day + 1, "ScanDuration": scan_duration, "PatientType": 2})

# Combine Type 1 and Type 2 data
simulated_df_type1 = pd.DataFrame(simulated_data)
simulated_df_type2 = pd.DataFrame(simulated_data_type2)
combined_simulated_df = pd.concat([simulated_df_type1, simulated_df_type2], ignore_index=True)

# Now combined_simulated_df contains data for both Type 1 and Type 2 patients.


In [45]:
# Combining the performance analysis for Type 1 and Type 2
combined_performance = []
for day in combined_simulated_df["Day"].unique():
    day_data = combined_simulated_df[combined_simulated_df["Day"] == day]
    total_scan_time = day_data["ScanDuration"].sum() * 60  # Convert hours to minutes
    
    # Check for overtime
    overtime = max(0, total_scan_time - total_available_time)  # Overtime in minutes
    utilization = total_scan_time / total_available_time  # Utilization as a fraction
    
    combined_performance.append({
        "Day": day,
        "Utilization (fraction)": utilization,
        "Overtime (minutes)": overtime
    })

# Convert to DataFrame for analysis
combined_performance_df = pd.DataFrame(combined_performance)

# Print summary of combined performance
print(combined_performance_df.describe())


             Day  Utilization (fraction)  Overtime (minutes)
count  30.000000               30.000000           30.000000
mean   15.500000                1.294620          194.097633
std     8.803408                0.421220          182.296680
min     1.000000                0.552818            0.000000
25%     8.250000                0.946965            0.000000
50%    15.500000                1.251894          136.022541
75%    22.750000                1.679011          366.665701
max    30.000000                1.896427          484.070372


In [48]:
combined_performance_df

,Day,Utilization (fraction),Overtime (minutes)
0,1,0.778542,0.000000
1,2,1.101926,55.039785
2,3,0.552818,0.000000
3,4,1.566490,305.904482
4,5,1.687750,371.385182
5,6,1.896427,484.070372
6,7,1.265387,143.309184
7,8,0.966550,0.000000
8,9,0.651229,0.000000
9,10,1.783875,423.292394


In [46]:
# Monte Carlo simulations for Type 2 uncertainty analysis
num_simulations = 50
monte_carlo_results = []

for _ in range(num_simulations):
    simulated_data = []
    for day in range(num_days):
        daily_arrivals_type2 = np.random.normal(mean_arrivals_type2, std_arrivals_type2)
        daily_arrivals_type2 = max(0, int(np.round(daily_arrivals_type2)))
        
        # Generate scan durations using chosen distributions
        if np.random.choice([True, False]):
            daily_durations_type2 = np.random.gamma(mean_duration_type2, std_duration_type2, daily_arrivals_type2)
        else:
            daily_durations_type2 = np.random.lognormal(np.log(mean_duration_type2), std_duration_type2, daily_arrivals_type2)
        
        # Store simulated data
        for scan_duration in daily_durations_type2:
            simulated_data.append({"ScanDuration": scan_duration})
    
    # Analyze the simulated data for this iteration
    simulated_df = pd.DataFrame(simulated_data)
    mean_duration = simulated_df["ScanDuration"].mean()
    variance_duration = simulated_df["ScanDuration"].var()
    
    # Store results
    monte_carlo_results.append({"Mean": mean_duration, "Variance": variance_duration})

# Average over Monte Carlo simulations
mc_results_df = pd.DataFrame(monte_carlo_results)
print(mc_results_df.describe())


            Mean   Variance
count  50.000000  50.000000
mean    0.314335   0.064142
std     0.043560   0.003323
min     0.211104   0.054386
25%     0.293218   0.062763
50%     0.316899   0.064575
75%     0.338518   0.066818
max     0.406347   0.069871


In [47]:
mc_results_df

,Mean,Variance
0,0.242157,0.060255
1,0.261632,0.064831
2,0.336872,0.061389
3,0.352141,0.066031
4,0.225287,0.054386
5,0.312638,0.067935
6,0.235401,0.064743
7,0.327384,0.063612
8,0.211104,0.056162
9,0.319166,0.064734
